In [64]:
import os
import pandas as pd
from pandas import read_csv, read_excel, DataFrame
from sae import COTSCar
import gurobipy as gp
from gurobipy import GRB
from gurobipy import quicksum as qsum
import numpy as np
from typing import List

import sys
sys.path.append('../')
import saedfsc

Price of the car.

In [65]:
maxProductPrice = 400000

Information about customers

In [66]:
customers = saedfsc.customers['Name'].to_list()
cQty = dict(zip(saedfsc.customers['Name'], saedfsc.customers['Quantity'])) # customer quantities
cPriceFocus = dict(zip(saedfsc.customers['Name'], saedfsc.customers['PriceFocus']))
name_weights_dict = saedfsc.customers.set_index('Name')['PerformanceUtilityWeights'].to_dict()
cWts = {c : np.fromstring(name_weights_dict[c].strip('[]'), sep=',') for c in name_weights_dict}

Methods to help compute the market share for each customer type (based on logit model).

In [67]:
def getTotalUtilityForCustomer(car : COTSCar, c : str, utilFn = 'partworth'):
    total_utility = 0
    perf_utility = car.partworth_objectives(weights=cWts[c])[0]
    pricePerf = (maxProductPrice - car.price) / maxProductPrice # Normalized price (0 is best, 1 is worst)
    total_utility += (1-cPriceFocus[c])*perf_utility - cPriceFocus[c]*pricePerf
    return total_utility

def getMarketShare(car : COTSCar, c : str, competitors : List[COTSCar]): # based on logit model of demand
    carUtility = getTotalUtilityForCustomer(car, c)
    totalCompetitorUtility = sum([getTotalUtilityForCustomer(competitorCar, c) for competitorCar in competitors])
    return carUtility / (totalCompetitorUtility + carUtility)

Create a set of competitors.

In [68]:
profitMargin = 0.1
competitors = []
for i in range(3):
    competitorCar = COTSCar()
    competitorCar.price = (1+profitMargin)*competitorCar.cost()
    competitorCar.name = "Competitor Car " + str(i)
    competitors.append(competitorCar)

names = [car.name for car in competitors]
designs = [car.vector for car in competitors]
salesPrices = ['$' + format(car.price, ",.2f") for car in competitors]
costs = ['$' + format(car.cost(), ",.2f") for car in competitors]
competitorsDF = pd.DataFrame({
    'Name': names,
    'Design': designs,
    'SalesPrice': salesPrices,
    'ProductCost': costs
})
competitorsDF

,Name,Design,SalesPrice,ProductCost
0,Competitor Car 0,"[5, 4, 11, 10, 2, 5, 1, 17, 23, 1, 55, 20, 0, ...","$35,191.41","$31,992.19"
1,Competitor Car 1,"[9, 1, 11, 7, 0, 1, 1, 19, 13, 0, 63, 175, 0, ...","$90,234.28","$82,031.16"
2,Competitor Car 2,"[6, 6, 2, 1, 1, 4, 2, 1, 5, 1, 59, 177, 4, 1, ...","$6,246.79","$5,678.90"


Create a random car and compute market share

In [73]:
carDesignVec = [12, 7, 6, 12, 9, 1, 0, 13, 9, 1, 39, 26, 2, 1, 33, 11, 4]
product = COTSCar()
product.vector = carDesignVec
product.price = (1 + profitMargin)*product.cost()
print("OUR CAR:") 
print("Design: ", product.vector)
print("Sales price: $", format(product.price, ",.2f"))
print("Cost: $", format(product.cost(), ",.2f"))

qtys = [cQty[c] for c in customers]
utilities = [getTotalUtilityForCustomer(product, c) for c in customers]
market_shares = [getMarketShare(product, c, competitors) for c in customers]
expected_customer_demand = [getMarketShare(product, c, competitors)*cQty[c] for c in customers]

df = pd.DataFrame({
    'Name': customers,
    'Quantity': qtys,
    'Utility for our car': utilities,
    'Market share': market_shares,
    'Expected demand': expected_customer_demand
})
total_profit = product.price*df['Expected demand'].sum()
print("Total expected profit: $", format(total_profit, ",.2f"))
print("Market information for our car:")
df

OUR CAR:
Design:  [12, 7, 6, 12, 9, 1, 0, 13, 9, 1, 39, 26, 2, 1, 33, 11, 4]
Sales price: $ 46,058.10
Cost: $ 41,871.00
Total expected profit: $ 72,968,575.16
Market information for our car:


,Name,Quantity,Utility for our car,Market share,Expected demand
0,CustomerType1,207,-0.221405,0.211840,43.850893
1,CustomerType2,362,-0.592687,0.119552,43.277659
2,CustomerType3,56,-0.624390,0.150161,8.409005
3,CustomerType4,282,-0.330067,0.163141,46.005815
4,CustomerType5,622,-0.314065,0.098108,61.023315
5,CustomerType6,363,-0.253927,0.121036,43.936224
6,CustomerType7,985,-0.543148,0.088827,87.494893
7,CustomerType8,766,-0.377860,0.181624,139.123761
8,CustomerType9,116,-0.547721,0.127825,14.827727
9,CustomerType10,763,-0.539879,0.209381,159.757693
